# Backpropagation

## References

* _Rumelhart et al. 1986, Learning representations by back-propagating errors_, [nature](https://www.nature.com/articles/323533a0)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.datasets as sk_datasets
import torch
from sklearn import metrics

import random_neural_net_models.backprop_rumelhart as backprop_rumelhart

In [ ]:
SEED = 42

In [ ]:
X, y = sk_datasets.make_regression(
    n_samples=100, n_features=1, noise=20, random_state=SEED
)

In [ ]:
sns.scatterplot(x=X[:, 0], y=y)
plt.tight_layout()

In [ ]:
X, y = sk_datasets.make_blobs(
    n_samples=1_000,
    n_features=2,
    centers=2,
    random_state=SEED,
)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, alpha=0.3);

## Rumelhart et al. 1986, "Learning representations by back-propagating errors"
> The following is in the spirit of the paper, *cough*. Found the presentation in the paper somewhat hard to read.

With input $x$ we predict $y$ using a function $f$ like

$$

y = f(x)

$$

$f$ uses weight $w$ and some non-linearity $g$ like

$$
z =  x \cdot w^T \\
f(x) = g(z)
$$

In the paper they use the sigmoid
$$
g(z) = \frac{1}{1+\exp(-z)}
$$

Indicating repeating weight multiplication and non-linearity with index $i$ (layer), we can write

$$

z_1 =  x \cdot w_1 ^ T \\
a_1 = g_1(z_1) \\

...\\

z_{i} = a_{i-1} \cdot w_{i}^T \\
a_{i} = g_{i}(z_{i}) \\

...\\

z_N = a_{N-1} \cdot w_N^T \\
a_N = g_N(z_N) \\

f(x) = a_N

$$

where $a_i$ is "activation" of layer $i$

Comparing predictions to desired valued $d$ we denote deviations / the loss as $l(d,y)$.

With this we can identify how to change $w_i$ by how much lead to the strongest improvement of $l(d,y)$ for any $x$, $y$, $d$ by differentiating $l$. A straightforward way is scaling that gradient and applying it like

$$

w_{i,\text{new}} = w_{i,\text{old}} + \epsilon \frac{d}{dw_i}l(d,y)

$$

with some factor $\epsilon \in [0,1]$

$\frac{d}{dw_i}l(d,y)$ can analytically be derived using the chain rule as 

$$

\frac{d}{dw_i}l(d,y) = l^\prime(d,y) \cdot g_N^\prime(z_N) \cdot z_N^\prime \cdot g_{N-1}^\prime(z_{N-1}) \cdot z_{N-1}^\prime \cdot \space ... \space \cdot g_{i}^\prime(z_{i}) \cdot a_{i-1}

$$

Replacing $z^\prime = \frac{\partial}{\partial a}z = w ^ T$
$$

\frac{d}{dw_i}l(d,y) = l^\prime(d,y) \cdot g_N^\prime(z_N) \cdot w_N ^ T \cdot g_{N-1}^\prime(z_{N-1}) \cdot w_{N-1} ^ T \cdot \space ... \space \cdot g_{i}^\prime(z_{i}) \cdot a_{i-1}

$$

Another version proposed in Rumelhart et al. is
$$

w_{i,\text{new}} = w_{i,\text{old}} + \epsilon \frac{d}{dw_i}l(d,y) + \alpha \left( \frac{d}{dw_i}l(d,y) \right)_\text{old}

$$

with some factor $\alpha \in [0,1]$.

In [ ]:
x = torch.linspace(-10, 10, 100)
a = backprop_rumelhart.sigmoid(x)
a_prime = backprop_rumelhart.sigmoid_derivative(x)

In [ ]:
sns.lineplot(x=x, y=a, label="g(z): sigmoid")
sns.lineplot(x=x, y=a_prime, label="dg(z)/dz: sigmoid derivative")

In [ ]:
model = backprop_rumelhart.Rumelhart1986PerceptronClassifier(
    n_hidden=(10, 5), epochs=10, verbose=True, eps=1e-3, alpha=1e-3
)

In [ ]:
model.fit(X, y)

In [ ]:
model.errors_

In [ ]:
x0 = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
X0, X1 = np.meshgrid(x0, x1)
X_plot = np.array([X0.ravel(), X1.ravel()]).T
X_plot[:4]

In [ ]:
y_prob = model.predict_proba(X_plot)

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(X0, X1, y_prob.reshape(X0.shape), alpha=0.2)
fig.colorbar(im, ax=ax)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, alpha=0.3)
plt.show()

In [ ]:
y_pred = model.predict(X)

In [ ]:
print(metrics.classification_report(y, y_pred))